<a href="https://colab.research.google.com/github/JulianCarax01/Fondamenti-web-app/blob/main/DijkstraSerialCPPMerge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ncu --mode launch-and-attach -o profile --target-processes all --nvtx --call-stack --section ComputeWorkloadAnalysis --section MemoryWorkloadAnalysis -f ./dijkstra_serialMerge.o


==ERROR== './dijkstra_serialMerge.o' does not exist or is not an executable. Please make sure to specify the absolute path to './dijkstra_serialMerge.o' if the executable is not in the local directory.


In [60]:
%%writefile dijkstra_serialMerge_3kern.cu
#include <iostream>
#include <vector>
#include <cstdlib>
#include <climits>
#include <cuda.h>
#include <curand_kernel.h>
#include <chrono>

// Funzione per eseguire Dijkstra in modo seriale sulla CPU
void dijkstraSerial(const int *graph, int n, int start, int *distances) {
    // Array per tenere traccia dei nodi visitati
    std::vector<int> visited(n, 0);

    // Inizializza le distanze
    for (int i = 0; i < n; ++i) {
        distances[i] = (i == start) ? 0 : INT_MAX;
    }

    for (int i = 0; i < n; ++i) {
        int minDist = INT_MAX;
        int currentNode = -1;

        // Trova il nodo con la distanza minima non ancora visitato
        for (int j = 0; j < n; ++j) {
            if (!visited[j] && distances[j] < minDist) {
                minDist = distances[j];
                currentNode = j;
            }
        }

        if (currentNode == -1) break; // Nessun altro nodo raggiungibile

        // Marca il nodo corrente come visitato
        visited[currentNode] = 1;

        // Aggiorna le distanze per i nodi adiacenti
        for (int j = 0; j < n; ++j) {
            int weight = graph[currentNode * n + j]; // Peso dell'arco currentNode -> j
            if (weight > 0 && weight != INT_MAX) {  // Esiste un arco valido
                // Verifica per evitare overflow: dist[currentNode] + weight
                if (distances[currentNode] != INT_MAX &&
                    distances[currentNode] + weight < distances[j]) {
                    distances[j] = distances[currentNode] + weight;
                }
            }
        }
    }
}

// Kernel CUDA per aggiornare le distanze
__global__ void cudaNodeRelax(int nodo, int *graph, int *distances, int *visited, int n) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;

    // Carica la riga del nodo corrente nella memoria condivisa
    extern __shared__ int sharedRow[];
    if (tid < n) {
        sharedRow[tid] = graph[nodo * n + tid];
    }
    __syncthreads();

    if (tid < n && visited[tid] == 0) { // Assicurati che il nodo non sia stato visitato
        int weight = sharedRow[tid]; // Peso dell'arco (nodo -> tid)
        if (weight != INT_MAX && weight != 0) { // Arco esistente e diverso da se stesso
            atomicMin(&distances[tid], distances[nodo] + weight); // Aggiorna la distanza con atomicMin
        }
    }
}

__global__ void findMinReductionCUDA(int *distanze, int *visitato, int n, int *minNodo, int *minDistanza) {
    // Memoria condivisa dinamica
    extern __shared__ int sharedMem[];

    // Dividi la memoria condivisa in due array distinti
    int *sharedDistanze = sharedMem;                           // Primo array: memorizza le distanze
    int *sharedNodi = &sharedDistanze[n];                     // Secondo array: memorizza gli indici dei nodi

    int tid = threadIdx.x;                   // Indice del thread locale al blocco
    int gid = blockIdx.x * blockDim.x + tid; // Indice globale del thread

    // Inizializzazione della memoria condivisa
    if (gid < n && tid < blockDim.x) {
        sharedDistanze[tid] = (visitato[gid] == 0) ? distanze[gid] : INT_MAX;
        sharedNodi[tid] = gid;
    } else {
        sharedDistanze[tid] = INT_MAX;
        sharedNodi[tid] = -1;
    }
    __syncthreads();

    // Riduzione parallela per trovare il minimo
    for (int stride = blockDim.x / 2; stride > 0; stride /= 2) {
        if (gid < n && tid + stride < blockDim.x) {
            if (sharedDistanze[tid + stride] < sharedDistanze[tid]) {
                sharedDistanze[tid] = sharedDistanze[tid + stride];
                sharedNodi[tid] = sharedNodi[tid + stride];
            }
        }
        __syncthreads();
    }

    // Il thread 0 del blocco salva il risultato parziale in modo atomico
    if (tid == 0) {
        atomicExch(minNodo, sharedNodi[0]);
        atomicExch(minDistanza, sharedDistanze[0]);
        atomicExch(&visitato[sharedNodi[0]], 1); // Segna come visitato
    }
}


__global__ void initialValueKernel(int*distances, int*visited,int start,int n){
  int id = threadIdx.x + blockIdx.x * blockDim.x;
    if (id < n) {
        distances[id] = (id == start) ? 0 : INFINITY;
        visited[id] = 0;
    }
}

void dijkstraCUDA(int *graph, int n, int start, int *distances)  {
    // Allocazione memoria su GPU
    int *d_graph,*d_minNodo, *d_minDistanza, *d_distances, *d_visited;

    // Allocazione memoria sulla GPU
    cudaMalloc((void **)&d_graph, n * n * sizeof(int));
    cudaMalloc((void **)&d_distances, n * sizeof(int));
    cudaMalloc((void **)&d_visited, n * sizeof(int));
    cudaMalloc(&d_minNodo, sizeof(int));
    cudaMalloc(&d_minDistanza, sizeof(int));

    int blockSize = n;
    int gridDim=(n + blockSize - 1) / blockSize;
    initialValueKernel<<<gridDim,blockSize>>>(d_distances,d_visited,start,n);
    cudaDeviceSynchronize();

    // Copia il grafo e i dati iniziali sulla GPU
    cudaMemcpy(d_graph, graph, n * n * sizeof(int), cudaMemcpyHostToDevice);

    for (int i = 0; i < n; ++i) {
        // Resetta `minDistanza` al massimo valore possibile
        int h_minDistanza = INT_MAX;
        cudaMemcpy(d_minDistanza, &h_minDistanza, sizeof(int), cudaMemcpyHostToDevice);

        // Trova il nodo con la distanza minima non visitato
        blockSize = n;
        gridDim=(n + blockSize - 1) / blockSize;
        findMinReductionCUDA<<<gridDim, blockSize, blockSize * sizeof(int) * 2>>>(d_distances, d_visited, n, d_minNodo, d_minDistanza);

        if (cudaDeviceSynchronize() != cudaSuccess) {
        std::cerr << "Errore durante l'esecuzione del kernel." << std::endl;
        return;
        }

        // Recupera il nodo con distanza minima dalla GPU
        int h_minNodo;
        cudaMemcpy(&h_minNodo, d_minNodo, sizeof(int), cudaMemcpyDeviceToHost);
        cudaMemcpy(&h_minDistanza, d_minDistanza, sizeof(int), cudaMemcpyDeviceToHost);
        // Se non ci sono più nodi raggiungibili, esci
        if (h_minNodo == -1 || h_minDistanza == INT_MAX) {
            break;
        }
        cudaNodeRelax<<<gridDim, blockSize, blockSize * sizeof(int)>>>(h_minNodo, d_graph, d_distances, d_visited, n);
        cudaDeviceSynchronize();
    }

    // Copia i risultati dalla GPU alla CPU
    cudaMemcpy(distances, d_distances, n * sizeof(int), cudaMemcpyDeviceToHost);

    // Libera memoria GPU
    cudaFree(d_graph);
    cudaFree(d_distances);
    cudaFree(d_visited);
    cudaFree(d_minNodo);
    cudaFree(d_minDistanza);
}



__global__ void generateGraphKernel(int *graph, int n, int minWeight, int maxWeight, unsigned int seed) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;

    // Shared memory per una porzione della matrice
    extern __shared__ int sharedRow[];

    // Stato del generatore casuale
    curandState state;

    // Grid-stride loop per coprire tutta la matrice
    for (int i = idx; i < n * n; i += blockDim.x * gridDim.x) {
        int row = i / n; // Calcola la riga
        int col = i % n; // Calcola la colonna

        // Inizializza lo stato casuale
        curand_init(seed + i, 0, 0, &state);

        if (threadIdx.x < n) {
            sharedRow[threadIdx.x] = graph[row * n + threadIdx.x];
        }
        __syncthreads();

        // Calcola il peso per la triangolare superiore
        int weight = (row < col)
                     ? ((curand(&state) % 100 < 40) ? curand(&state) % (maxWeight - minWeight + 1) + minWeight : INT_MAX)
                     : 0;

        // Scrive nella memoria globale
        if (row < col) {
            graph[row * n + col] = weight;
            graph[col * n + row] = weight; // Simmetria
        }

        // Imposta la diagonale a 0
        if (row == col) {
            graph[row * n + col] = 0;
        }
        __syncthreads();
    }
}




void generateGraphCUDA(int *graph, int n, int minWeight, int maxWeight) {
    int *d_graph;
    size_t size = n * n * sizeof(int);

    // Allocazione della memoria sulla GPU
    if (cudaMalloc(&d_graph, size) != cudaSuccess) {
        std::cerr << "Errore nell'allocazione della memoria sulla GPU." << std::endl;
        return;
    }
    if (cudaMemset(d_graph, 0, size) != cudaSuccess) {
        std::cerr << "Errore durante l'inizializzazione della memoria sulla GPU." << std::endl;
        cudaFree(d_graph);
        return;
    }

    // Configura blocchi e griglia con valori fissi
    dim3 threadsPerBlock(1024, 1, 1);
    dim3 blocksPerGrid((n + threadsPerBlock.x - 1) / threadsPerBlock.x, 1, 1);


    // Calcola la dimensione della shared memory per blocco
    size_t sharedMemorySize = threadsPerBlock.x * sizeof(int);

    // Lancia il kernel con configurazione fissa
    generateGraphKernel<<<blocksPerGrid, threadsPerBlock, sharedMemorySize>>>(d_graph, n, minWeight, maxWeight, time(NULL));

    if (cudaDeviceSynchronize() != cudaSuccess) {
        std::cerr << "Errore durante l'esecuzione del kernel." << std::endl;
        cudaFree(d_graph);
        return;
    }

    // Copia i dati dalla GPU alla CPU
    if (cudaMemcpy(graph, d_graph, size, cudaMemcpyDeviceToHost) != cudaSuccess) {
        std::cerr << "Errore nella copia dei dati dalla GPU alla CPU." << std::endl;
        cudaFree(d_graph);
        return;
    }

    // Libera memoria GPU
    cudaFree(d_graph);
}


void generateGraphSerial(int *graph, int n, int minWeight, int maxWeight) {
    // Inizializza il generatore di numeri casuali
    std::srand(std::time(0));

    for (int row = 0; row < n; ++row) {
        for (int col = 0; col < n; ++col) {
            if (row == col) {
                // Imposta la diagonale principale a 0
                graph[row * n + col] = 0;
            } else if (row < col) {
                // Genera un arco casuale per la triangolare superiore
                int randomValue = std::rand() % 100;
                if (randomValue < 40) { // 40% probabilità di avere un arco
                    int weight = std::rand() % (maxWeight - minWeight + 1) + minWeight;
                    graph[row * n + col] = weight;
                    graph[col * n + row] = weight; // Simmetria
                } else {
                    graph[row * n + col] = INT_MAX; // Nessun arco
                    graph[col * n + row] = INT_MAX; // Nessun arco
                }
            }
        }
    }
}


int main() {
    int n = 16384; // Numero di nodi
    int start = 0; // Nodo iniziale
    int minWeight = 1;
    int maxWeight = 2000;

    std::vector<int> graph(n * n);

// Generazione del grafo con CUDA
auto startTime = std::chrono::high_resolution_clock::now();
generateGraphCUDA(graph.data(), n, minWeight, maxWeight);
auto endTime = std::chrono::high_resolution_clock::now();
long cudaGraphTime = std::chrono::duration_cast<std::chrono::microseconds>(endTime - startTime).count();

// Generazione del grafo Seriale
auto startTimeS = std::chrono::high_resolution_clock::now();
generateGraphSerial(graph.data(), n, minWeight, maxWeight);
auto endTimeS = std::chrono::high_resolution_clock::now();
long cudaGraphTimeS = std::chrono::duration_cast<std::chrono::microseconds>(endTimeS - startTimeS).count();

// Calcolo delle distanze con Dijkstra Seriale
std::vector<int> serialDistances(n);
startTime = std::chrono::high_resolution_clock::now();
dijkstraSerial(graph.data(), n, start, serialDistances.data());
endTime = std::chrono::high_resolution_clock::now();
long serialTime = std::chrono::duration_cast<std::chrono::microseconds>(endTime - startTime).count();

// Calcolo delle distanze con Dijkstra CUDA
std::vector<int> cudaDistances(n);
startTime = std::chrono::high_resolution_clock::now();
dijkstraCUDA(graph.data(), n, start, cudaDistances.data());
endTime = std::chrono::high_resolution_clock::now();
long cudaTime = std::chrono::duration_cast<std::chrono::microseconds>(endTime - startTime).count();

/*
// Stampa del grafo
std::cout << "=== Matrice del grafo ===\n";
for (int i = 0; i < n; ++i) {
    for (int j = 0; j < n; ++j) {
        if (i == j) {
            std::cout << "X "; // Diagonale principale
        } else if (graph[i * n + j] == INT_MAX) {
            std::cout << "_ "; // Nessun collegamento
        } else {
            std::cout << graph[i * n + j] << " "; // Peso dell'arco
        }
    }
    std::cout << "\n";
}

// Confronto distanze Seriali vs CUDA
std::cout << "\n=== Distanze (Seriale vs CUDA) ===\n";
std::cout << "Nodo\tSeriale\tCUDA\n";
for (int i = 0; i < n; ++i) {
    std::cout << i << "\t";
    if (serialDistances[i] == INT_MAX)
        std::cout << "∞\t";
    else
        std::cout << serialDistances[i] << "\t";

    if (cudaDistances[i] == INT_MAX)
        std::cout << "∞\n";
    else
        std::cout << cudaDistances[i] << "\n";
}
*/

// Stampa dei tempi di generazione e di esecuzione
std::cout << "\n=== Tempi di generazione ===\n";
std::cout << "Tempo di generazione Seriale: " << cudaGraphTimeS << " µs\n";
std::cout << "Tempo di generazione CUDA: " << cudaGraphTime << " µs\n";

std::cout << "\n=== Tempi di esecuzione Dijkstra ===\n";
std::cout << "Tempo esecuzione Seriale: " << serialTime << " µs\n";
std::cout << "Tempo esecuzione CUDA: " << cudaTime << " µs\n";

return 0;
}


Overwriting dijkstra_serialMerge_3kern.cu


In [61]:
!nvcc -rdc=true dijkstra_serialMerge_3kern.cu -lcudadevrt -o dijkstra_serialMerge_3kern.o


In [62]:
!nvprof ./dijkstra_serialMerge_3kern.o

==9009== NVPROF is profiling process 9009, command: ./dijkstra_serialMerge_3kern.o

=== Tempi di generazione ===
Tempo di generazione Seriale: 13408282 µs
Tempo di generazione CUDA: 551558 µs

=== Tempi di esecuzione Dijkstra ===
Tempo esecuzione Seriale: 4141141 µs
Tempo esecuzione CUDA: 231516 µs
==9009== Profiling application: ./dijkstra_serialMerge_3kern.o
==9009== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   41.40%  227.37ms         2  113.69ms  1.1840us  227.37ms  [CUDA memcpy HtoD]
                   41.26%  226.56ms         4  56.640ms  1.8880us  226.55ms  [CUDA memcpy DtoH]
                   16.50%  90.620ms         1  90.620ms  90.620ms  90.620ms  generateGraphKernel(int*, int, int, int, unsigned int)
                    0.84%  4.6117ms         1  4.6117ms  4.6117ms  4.6117ms  [CUDA memset]
      API calls:   70.14%  454.63ms         6  75.771ms  11.713us  227.55ms  cudaMemcpy
                   14.49%

In [ ]:
!ncu --mode launch-and-attach -o profile --target-processes all --nvtx --call-stack --section ComputeWorkloadAnalysis --section MemoryWorkloadAnalysis -f ./dijkstra_serialMerge_3kern.o

==PROF== Connected to process 13064 (/content/dijkstra_serialMerge_3kern.o)
==PROF== Profiling "generateGraphKernel" - 0: 0%..
==WARNING== Launching the workload is taking more time than expected. If this continues to hang, terminate the profile and re-try by profiling the range of all related launches using '--replay-mode range'. See https://docs.nvidia.com/nsight-compute/ProfilingGuide/index.html#replay for more details.
..50%....100% - 8 passes

=== Tempi di generazione ===
Tempo di generazione Seriale: 73184554 µs
Tempo di generazione CUDA: 30554133 µs

=== Tempi di esecuzione Dijkstra ===
Tempo esecuzione Seriale: 20424244 µs
Tempo esecuzione CUDA: 975253 µs
==PROF== Disconnected from process 13064
==PROF== Report: /content/profile.ncu-rep


In [ ]:
!ncu --target-processes=all ./dijkstra_serialMerge_3kern.o

==PROF== Connected to process 14359 (/content/dijkstra_serialMerge_3kern.o)
==PROF== Profiling "generateGraphKernel" - 0: 0%....50%....100% - 8 passes
==PROF== Profiling "initialValueKernel" - 1: 0%....50%....100% - 8 passes
==PROF== Profiling "findMinReductionCUDA" - 2: 0%....50%....100% - 8 passes
==PROF== Profiling "cudaNodeRelax" - 3: 0%....50%....100% - 8 passes
==PROF== Profiling "findMinReductionCUDA" - 4: 0%....50%....100% - 8 passes
==PROF== Profiling "cudaNodeRelax" - 5: 0%....50%....100% - 8 passes
==PROF== Profiling "findMinReductionCUDA" - 6: 0%....50%....100% - 8 passes
==PROF== Profiling "cudaNodeRelax" - 7: 0%....50%....100% - 8 passes
==PROF== Profiling "findMinReductionCUDA" - 8: 0%....50%....100% - 8 passes
==PROF== Profiling "cudaNodeRelax" - 9: 0%....50%....100% - 8 passes
==PROF== Profiling "findMinReductionCUDA" - 10: 0%....50%....100% - 8 passes
==PROF== Profiling "cudaNodeRelax" - 11: 0%....50%....100% - 8 passes
==PROF== Profiling "findMinReductionCUDA" - 12: 

In [ ]:
!./dijkstra_serialMerge_3kern.o

In [ ]:
%%writefile debug.cu
#include <iostream>
#include <climits>
#include <cuda_runtime.h>

// Funzione per simulare la conversione da matrice 2D a array 1D
void convertMatrixTo1D(const int matrix[5][17], int *graph, int n) {
    for (int i = 0; i < n; ++i) {
        for (int j = 0; j < n; ++j) {
            if (matrix[i][j] == -1) { // Se elemento è '-', lo convertiamo in INT_MAX
                graph[i * n + j] = INT_MAX;
            } else {
                graph[i * n + j] = matrix[i][j];
            }
        }
    }
}

// Kernel per rilassare gli archi adiacenti (debug con printf)
__global__ void cudaNodeRelax(int nodo, int *graph, int *distances, int *visited, int n) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;

    extern __shared__ int sharedRow[];

    if (tid < n) {
        sharedRow[tid] = graph[nodo * n + tid];
        printf("Thread %d: Carico nella memoria condivisa sharedRow[%d] = %d\n",
               tid, tid, sharedRow[tid]);
    }
    __syncthreads();

    if (tid < n && visited[tid] == 0) {
        int weight = sharedRow[tid];
        if (weight != INT_MAX && weight != 0) {
            printf("Thread %d: Aggiorno con peso %d, distanza attuale %d\n",
                   tid, weight, distances[tid]);
            atomicMin(&distances[tid], distances[nodo] + weight);
            printf("Thread %d: Distanza aggiornata a %d\n", tid, distances[tid]);
        }
    }
    __syncthreads();
}

// Kernel per ridurre le distanze con debug
__global__ void findMinReductionCUDA(int *distanze, int *visitato, int n, int *minNodo, int *minDistanza) {
    extern __shared__ int sharedMem[];

    int *sharedDistanze = sharedMem;
    int *sharedNodi = &sharedDistanze[n];

    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + tid;

    if (gid < n) {
        sharedDistanze[tid] = (visitato[gid] == 0) ? distanze[gid] : INT_MAX;
        sharedNodi[tid] = gid;
        printf("Thread %d: Caricamento sharedDistanze[%d] = %d\n",
               tid, tid, sharedDistanze[tid]);
    } else {
        sharedDistanze[tid] = INT_MAX;
        sharedNodi[tid] = -1;
    }

    __syncthreads();

    for (int stride = blockDim.x / 2; stride > 0; stride /= 2) {
        if (tid < stride) {
            if (sharedDistanze[tid + stride] < sharedDistanze[tid]) {
                printf("Thread %d: Riduzione stride %d, sharedDistanze[%d] = %d aggiornato con %d\n",
                       tid, stride, tid, sharedDistanze[tid], sharedDistanze[tid + stride]);
                sharedDistanze[tid] = sharedDistanze[tid + stride];
                sharedNodi[tid] = sharedNodi[tid + stride];
            }
        }
        __syncthreads();
    }

    if (tid == 0) {
        printf("Riduzione completata: minDistanza = %d, nodo connesso = %d\n",
               sharedDistanze[0], sharedNodi[0]);
        atomicMin(minDistanza, sharedDistanze[0]);
        atomicExch(minNodo, sharedNodi[0]);
    }
}

__global__ void initialValueKernel(int*distances, int*visited,int start,int n){
  int id = threadIdx.x + blockIdx.x * blockDim.x;
    if (id < n) {
        distances[id] = (id == start) ? 0 : INFINITY;
        visited[id] = 0;
    }
}

int main() {
  const int start=0;
    const int n = 36;
    int matrix[5][17] = {
        { 0, -1, -1, -1, 1892, -1, 149, 1802, -1, -1, -1, -1, -1, -1, 1826, -1, 337 },
        { -1, 0, 1501, -1, -1, -1, 1061, 388, -1, 394, -1, -1, 1880, 1745, -1, -1, 1114 },
        { -1, 1501, 0, 1532, -1, -1, -1, 1454, 1362, -1, 849, -1, 605, -1, 1790, 408, -1 },
        { -1, -1, 1532, 0, -1, -1, 529, -1, 212, -1, 725, -1, -1, 620, -1, 218, -1 },
        { 1892, -1, -1, -1, 0, -1, -1, -1, -1, -1, 1956, 620, -1, -1, -1, 1834, 1304 }
    };

    int *d_graph,*d_minNodo, *d_minDistanza, *d_distances, *d_visited;

    // Allocazione memoria sulla GPU
    cudaMalloc((void **)&d_graph, n * n * sizeof(int));
    cudaMalloc((void **)&d_distances, n * sizeof(int));
    cudaMalloc((void **)&d_visited, n * sizeof(int));
    cudaMalloc(&d_minNodo, sizeof(int));
    cudaMalloc(&d_minDistanza, sizeof(int));

    int hostGraph[n * n];
    convertMatrixTo1D(matrix, hostGraph, n);
    cudaMemcpy(d_graph, hostGraph, n * n * sizeof(int), cudaMemcpyHostToDevice);

    int blockSize = n;
    int gridDim=(n + blockSize - 1) / blockSize;
    initialValueKernel<<<gridDim,blockSize>>>(d_distances,d_visited,start,n);
    cudaDeviceSynchronize();
    int sharedMemSize = 2 * n * sizeof(int);

    // Esegui findMinReduction
    for(int i=0;i<n;i++){
    // Resetta `minDistanza` al massimo valore possibile
        int h_minDistanza = INT_MAX;
        cudaMemcpy(d_minDistanza, &h_minDistanza, sizeof(int), cudaMemcpyHostToDevice);

        // Trova il nodo con la distanza minima non visitato
        blockSize = n;
        gridDim=(n + blockSize - 1) / blockSize;
        findMinReductionCUDA<<<gridDim, blockSize, blockSize * sizeof(int) * 2>>>(d_distances, d_visited, n, d_minNodo, d_minDistanza);

        if (cudaDeviceSynchronize() != cudaSuccess) {
        std::cerr << "Errore durante l'esecuzione del kernel." << std::endl;
        return;
        }

        // Recupera il nodo con distanza minima dalla GPU
        int h_minNodo;
        cudaMemcpy(&h_minNodo, d_minNodo, sizeof(int), cudaMemcpyDeviceToHost);
        cudaMemcpy(&h_minDistanza, d_minDistanza, sizeof(int), cudaMemcpyDeviceToHost);
        // Se non ci sono più nodi raggiungibili, esci
        if (h_minNodo == -1 || h_minDistanza == INT_MAX) {
            break;
        }
        cudaNodeRelax<<<gridDim, blockSize, blockSize * sizeof(int)>>>(h_minNodo, d_graph, d_distances, d_visited, n);
        cudaDeviceSynchronize();
    }

    int h_minNodo, h_minDistanza;
    cudaMemcpy(&h_minNodo, d_minNodo, sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(&h_minDistanza, d_minDistanza, sizeof(int), cudaMemcpyDeviceToHost);

    printf("Nodo con distanza minima trovato: %d, distanza = %d\n", h_minNodo, h_minDistanza);

    cudaFree(d_graph);
    cudaFree(d_minNodo);
    cudaFree(d_minDistanza);
    cudaFree(d_distances);

    return 0;
}

Overwriting debug.cu


In [ ]:
!nvcc -G -g debug.cu -o debug.o
! ./debug.o

debug.cu(120): warning #177-D: variable "sharedMemSize" was declared but never referenced
      int sharedMemSize = 2 * n * sizeof(int);
          ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

Thread 32: Caricamento sharedDistanze[32] = 2147483647
Thread 33: Caricamento sharedDistanze[33] = 2147483647
Thread 34: Caricamento sharedDistanze[34] = 2147483647
Thread 35: Caricamento sharedDistanze[35] = 2147483647
Thread 0: Caricamento sharedDistanze[0] = 0
Thread 1: Caricamento sharedDistanze[1] = 2147483647
Thread 2: Caricamento sharedDistanze[2] = 2147483647
Thread 3: Caricamento sharedDistanze[3] = 2147483647
Thread 4: Caricamento sharedDistanze[4] = 2147483647
Thread 5: Caricamento sharedDistanze[5] = 2147483647
Thread 6: Caricamento sharedDistanze[6] = 2147483647
Thread 7: Caricamento sharedDistanze[7] = 2147483647
Thread 8: Caricamento sharedDistanze[8] = 2147483647
Thread 9: Caricamento sharedDistanze[9] = 2147483647
Thread 10: Caricamento sharedD